In [ ]:
#PCA+keras

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('keystroke/userdata/data.csv')

In [7]:
len(data.sample(frac=1).reset_index(drop=True))

2981

In [10]:
import glob
data = pd.DataFrame(columns=['keycode','HD','PPD','RPD', 'RRD', 'user'])
for file in glob.glob("keystroke/userdata/processed_data/*_processed_data.csv"):
    print('[+]:', file)
    tmp = pd.read_csv(file)
    data = pd.concat([data, tmp], ignore_index=True)

data.to_csv('keystroke/userdata/data.csv', index=False) 

[+]: keystroke/userdata/processed_data\Alexander_processed_data.csv
[+]: keystroke/userdata/processed_data\Alex_processed_data.csv
[+]: keystroke/userdata/processed_data\Artem_processed_data.csv
[+]: keystroke/userdata/processed_data\DDD_processed_data.csv
[+]: keystroke/userdata/processed_data\Imya_processed_data.csv
[+]: keystroke/userdata/processed_data\nevova_processed_data.csv
[+]: keystroke/userdata/processed_data\Nikita_processed_data.csv
[+]: keystroke/userdata/processed_data\Sasha_processed_data.csv


In [3]:
test_data=pd.read_csv('server/keystroke/Nikita_test_logs.csv')
df = pd.DataFrame(columns=['keycode','HD','PPD','RPD', 'RRD', 'user'])

user = test_data.user[0]
event_len = len(test_data)

for i in range(event_len-3):
    if str(test_data.iloc[i].event) == 'Down':
        finalData = {}
        cur_key_press_time = test_data.iloc[i].time
        cur_key_release_time =  test_data[(test_data.keycode==test_data.iloc[i].keycode)&(test_data.event=='Up')&(test_data.time>test_data.iloc[i].time)].iloc[0].time
        next_key_press_time = test_data[(test_data.event=='Down')&(test_data.time>test_data.iloc[i].time)].iloc[0].time
        next_key_keycode = test_data[(test_data.event=='Down')&(test_data.time>test_data.iloc[i].time)].iloc[0].keycode
        next_key_release_time =  test_data[(test_data.keycode==next_key_keycode)&(test_data.event=='Up')&(test_data.time>next_key_press_time)].iloc[0].time
        finalData['keycode'] = test_data.iloc[i].keycode
        finalData['HD'] = cur_key_release_time-cur_key_press_time
        finalData['PPD'] = next_key_press_time-cur_key_press_time
        finalData['RPD'] = next_key_press_time-cur_key_release_time
        finalData['RRD'] = next_key_release_time-cur_key_release_time
        finalData['user'] = user
        df = df.append(finalData,ignore_index=True)

df.keycode = df.keycode.apply(int).apply(str) 

In [72]:
from sklearn import preprocessing
from sklearn.svm import OneClassSVM
import numpy as np



nu = np.arange(0.01, 1, 0.01)
gamma = np.arange(0.01, 1, 0.0025)
max = 0
for n in nu:
    for g in gamma:
        dif_len_rate = 0
        i = 0
        for l in range(40, 60, 3):
            n_scaler = preprocessing.Normalizer()
            data = pd.read_csv('server/keystroke/userdata/processed_data/Nikita_processed_data.csv')[:l]
            col_names = ['HD', 'PPD', 'RPD', 'RRD']
            data_s = n_scaler.fit_transform(data[col_names])
            data_s = pd.DataFrame(data_s, columns = col_names)
            data_s['keycode']=data.keycode.astype('str')
            clf = OneClassSVM(nu=n, kernel="rbf", gamma=g)
            #clf = SGDOneClassSVM(nu=0.5)
            clf.fit(data_s)

            Alex_data = pd.read_csv('server/keystroke/userdata/processed_data/Alex_processed_data.csv')[:20]
            Alex_data_s = n_scaler.transform(Alex_data[col_names])
            Alex_data_s = pd.DataFrame(Alex_data_s, columns = col_names)
            Alex_data_s['keycode']=Alex_data.keycode.astype('str')
            _, Alex_count =np.unique(clf.predict(Alex_data_s), return_counts=True)
            #print(f'Probability of valid user (Alex_1): {Alex_count[1]/20}')

            DDD_data = pd.read_csv('server/keystroke/userdata/processed_data/DDD_processed_data.csv')[:20]
            DDD_data_s = n_scaler.transform(DDD_data[col_names])
            DDD_data_s = pd.DataFrame(DDD_data_s, columns = col_names)
            DDD_data_s['keycode']=DDD_data.keycode.astype('str')
            _, DDD_count =np.unique(clf.predict(DDD_data_s), return_counts=True)
            #print(f'Probability of valid user (Dmitry): {DDD_count[1]/20}')

            Sasha_data = pd.read_csv('server/keystroke/userdata/processed_data/Sasha_processed_data.csv')[:20]
            Sasha_data_s = n_scaler.transform(Sasha_data[col_names])
            Sasha_data_s = pd.DataFrame(Sasha_data_s, columns = col_names)
            Sasha_data_s['keycode']=Sasha_data.keycode.astype('str')
            _, Sasha_count = np.unique(clf.predict(Sasha_data_s), return_counts=True)
            #print(f'Probability of valid user (Alex_2): {Sasha_count[1]/20}')

            Artem_data = pd.read_csv('server/keystroke/userdata/processed_data/Artem_processed_data.csv')[:20]
            Artem_data_s = n_scaler.transform(Artem_data[col_names])
            Artem_data_s = pd.DataFrame(Artem_data_s, columns = col_names)
            Artem_data_s['keycode']=Artem_data.keycode.astype('str')
            _, Artem_count = np.unique(clf.predict(Artem_data_s), return_counts=True)
            #print(f'Probability of valid user (Artem): {Artem_count[1]/20}')

            df_s = n_scaler.transform(df[col_names])
            df_s = pd.DataFrame(df_s, columns = col_names)
            df_s['keycode']=df.keycode.astype('str')
            _, test_valid_counts = np.unique(clf.predict(df_s[:20]), return_counts=True)
            #print(f'Probability of valid user (Nikita_test): {test_valid_counts[1]/20}')
            _, test_valid_counts_2 = np.unique(clf.predict(df_s[20:40]), return_counts=True)
            #print(f'Probability of valid user (Nikita_test_2): {test_valid_counts_2[1]/20}')
            try:
                false_rate = (Alex_count[1]+DDD_count[1]+Sasha_count[1]+Artem_count[1])/80
                true_rate = (test_valid_counts[1]+test_valid_counts_2[1])/40
                dif_len_rate+=(true_rate-false_rate)
                i+=1
            except:
                pass
        try:
            mean_rate = dif_len_rate/i
            if mean_rate>max:
                max = mean_rate
                max_n = n
                max_g = n
        except:
            pass           
            # if Alex_count[1]/20<0.5 and DDD_count[1]/20<0.5 and Sasha_count[1]/20<0.5 and Artem_count[1]/20<0.5 and test_valid_counts[1]/20>0.59 and test_valid_counts_2[1]/20>0.59:
            #     print(f'nu = {n}, gamma = {g}')
print(f'nu = {max_n}, gamma = {max_g}, max = {max}')        

nu = 0.33, gamma = 0.33, max = 0.2589285714285714


In [186]:
col_names = ['HD', 'PPD', 'RPD', 'RRD']
df_s = n_scaler.fit_transform(df[col_names])
df_s = pd.DataFrame(df_s, columns = col_names)
df_s['keycode']=df.keycode.astype('str')
print(np.unique(clf.predict(df_s[:25]), return_counts=True))

(array([-1,  1], dtype=int64), array([22,  3], dtype=int64))


In [192]:
np.unique(clf.predict(df_s[:20]), return_counts=True)[1][1]/20

0.7

In [78]:
from sklearn import preprocessing
from sklearn.svm import OneClassSVM
import numpy as np

n_scaler = preprocessing.Normalizer()
data = pd.read_csv('server/keystroke/userdata/processed_data/Nikita_processed_data.csv')[:49]
col_names = ['HD', 'PPD', 'RPD', 'RRD']
data_s = n_scaler.fit_transform(data[col_names])
data_s = pd.DataFrame(data_s, columns = col_names)
data_s['keycode']=data.keycode.astype('str')

# nu = np.arange(0.01, 1, 0.01)
# gamma = np.arange(0.01, 1, 0.01)
# for n in nu:
#     for g in gamma:
clf = OneClassSVM(nu=0.04, kernel="rbf", gamma=0.0275)
#clf = SGDOneClassSVM(nu=0.5)
clf.fit(data_s)

Alex_data = pd.read_csv('server/keystroke/userdata/processed_data/Alex_processed_data.csv')[:20]
Alex_data_s = n_scaler.transform(Alex_data[col_names])
Alex_data_s = pd.DataFrame(Alex_data_s, columns = col_names)
Alex_data_s['keycode']=Alex_data.keycode.astype('str')
_, Alex_count =np.unique(clf.predict(Alex_data_s), return_counts=True)
print(f'Probability of valid user (Alex_1): {Alex_count[1]/20}')

DDD_data = pd.read_csv('server/keystroke/userdata/processed_data/DDD_processed_data.csv')[:20]
DDD_data_s = n_scaler.transform(DDD_data[col_names])
DDD_data_s = pd.DataFrame(DDD_data_s, columns = col_names)
DDD_data_s['keycode']=DDD_data.keycode.astype('str')
_, DDD_count =np.unique(clf.predict(DDD_data_s), return_counts=True)
print(f'Probability of valid user (Dmitry): {DDD_count[1]/20}')

Sasha_data = pd.read_csv('server/keystroke/userdata/processed_data/Sasha_processed_data.csv')[:20]
Sasha_data_s = n_scaler.transform(Sasha_data[col_names])
Sasha_data_s = pd.DataFrame(Sasha_data_s, columns = col_names)
Sasha_data_s['keycode']=Sasha_data.keycode.astype('str')
_, Sasha_count = np.unique(clf.predict(Sasha_data_s), return_counts=True)
print(f'Probability of valid user (Alex_2): {Sasha_count[1]/20}')

Artem_data = pd.read_csv('server/keystroke/userdata/processed_data/Artem_processed_data.csv')[:20]
Artem_data_s = n_scaler.transform(Artem_data[col_names])
Artem_data_s = pd.DataFrame(Artem_data_s, columns = col_names)
Artem_data_s['keycode']=Artem_data.keycode.astype('str')
_, Artem_count = np.unique(clf.predict(Artem_data_s), return_counts=True)
print(f'Probability of valid user (Artem): {Artem_count[1]/20}')

df_s = n_scaler.transform(df[col_names])
df_s = pd.DataFrame(df_s, columns = col_names)
df_s['keycode']=df.keycode.astype('str')
_, test_valid_counts = np.unique(clf.predict(df_s[:20]), return_counts=True)
print(f'Probability of valid user (Nikita_test): {test_valid_counts[1]/20}')
_, test_valid_counts_2 = np.unique(clf.predict(df_s[20:40]), return_counts=True)
print(f'Probability of valid user (Nikita_test_2): {test_valid_counts_2[1]/20}')
        # if Alex_count[1]/20<0.5 and DDD_count[1]/20<0.5 and Sasha_count[1]/20<0.5 and Artem_count[1]/20<0.5 and test_valid_counts[1]/20>0.49 and test_valid_counts_2[1]/20>0.49:
        #     print(f'nu = {n}, gamma = {g}')     

Probability of valid user (Alex_1): 0.5
Probability of valid user (Dmitry): 0.55
Probability of valid user (Alex_2): 0.45
Probability of valid user (Artem): 0.6
Probability of valid user (Nikita_test): 0.85
Probability of valid user (Nikita_test_2): 0.8
